## Цель выявить закономерность между действиями и скрытыми картами оппонентов, предсказать силу комбинации
### На основе рекомендатеотных систем выявить типы оппонентов (колличесвто и их свойства - характерные действия относящие их к тому или иному типу\группе)
### На основе дерева решений и\или типа оппонентов предсказать силу комбинации
 - Преобраование данных в числовой тип
 - Добавление новых признаков (размер банка, размер ставки в четвертях <=1/3,<=1/2,<=2/3,<=3/4,<=1,>1, процент активности\агрессии из признаков активности\агрессии)
 - https://github.com/julianandrews/flopferret/blob/master/flopferret/board_texture.py

In [17]:
import re
import pandas as pd
from datetime import datetime
import numpy as np

import eval7, pprint
from phevaluator import evaluate_cards
from poker.room.pokerstars import PokerStarsHandHistory
from poker.hand import Hand, Combo
from poker.constants import Action, Position
from poker import handhistory

import matplotlib.pyplot as plt
plt.style.use('dark_background') #стиль графиков

import sentry_sdk

In [18]:
%%time

# Создали словарь
data_pd = {'HandId': [], 'Level': [], 'ante':[], 'BB':[], 'Players': [], 'Board':[], 'SHOW_DOWN':[],'pflop_chip_rais':[], 'pflop_chip_calls':[], 'flop_chip_bets':[], 'flop_chip_raises':[],
           'flop_chip_calls':[], 'turn_chip_bets':[], 'turn_chip_raises':[], 'turn_chip_calls':[], 'river_chip_bets':[], 'river_chip_raises':[], 'river_chip_calls':[],   
            'pflop_rais':[], 'pflop_calls':[], 'pflop_allin':[], 'flop_bets':[], 'flop_raises':[], 'flop_calls':[], 
           'flop_allin':[], 'turn_bets':[], 'turn_raises':[], 'turn_calls':[], 'turn_allin':[], 'river_bets':[], 'river_raises':[], 'river_calls':[], 'river_allin':[]} 


# Открываем файл
with open('dim100.txt', 'r',  encoding="utf8") as f: #  Poker_/613k.txt   14437.txt   G:\HandHistory\Bot_Hand100.txt
    data_r = f.read()

iterr = re.findall("\n\n\n", data_r).count('\n\n\n') # задаём колличесвто итераций
print(iterr)

data = ""
iter_SHOW = 0

for i in range(iterr+1): # фильтруем раздачи с ШД и без
    if data_r.split("\n\n\n")[i].count('SHOW DOWN'):
        data = data + data_r.split("\n\n\n")[i] + "\n\n\n"
        iter_SHOW += 1
    else:
        continue
print(iter_SHOW)  

# данные
result = re.findall('(\w+) Hand #(\d*): Tournament #(\d*),(.*) Level (.*) \(.*\/(.*)\) - (\d{4}/\d+/\d+) (\d+\D\d+\D\d+) (\w*)', data)
Tables = re.findall('Table \'.*\' (.*)-max Seat', data)
#bains = re.findall('(\D\d+\.\d+\+\D\d+\.\d+)', data)
#HandHiros = re.findall("Dealt to .* \D(.. ..)\D", data)
Boards = re.findall("Board \[(.*)\]", data)
#preflop_raises_ = re.findall("raises", data) # "raises"g
SHOW_DOWNs = re.findall("Seat \d+: .+? showed \[(.*)\] and won", data)
#SHOW_DOWN.group(1)

def preflop_acti(self):
    preflop_ = re.search("HOLE CARDS \*{3}\\n(.*)\*{3} FLOP", self, re.DOTALL)
    x = 0
    i = 0
    y = 0
    max_rais = 0 # максимальный рейз 
    sum_rais = 0 # сумма всех рейзов
    while i != preflop_.group(1).count('raises'):
        s = preflop_.group(1)
        if s.find('Dealt') == 0 and i == 0:
            y = 10
        if i != 0:
            y = x
        strart = s.find('to ', y)
        x = s.find('\n', strart)
        max_rais= s[strart+3:x]
        if max_rais.find(' ') > 0:
            max_rais = s[strart+3:x-14]
        i += 1
        x = x + 1
        sum_rais += int(max_rais)
        
    z1 = 0
    i1 = 0
    x1 = 0
    sum_call = 0
    while i1 != preflop_.group(1).count('calls'):
        s = preflop_.group(1)
        strart = s.find('calls ',x1)
        x1 = s.find('\n', strart)
        z1= s[strart+6:x1]
        if z1.find(' ') > 0:
            z1 = s[strart+6:x1-14]
        i1 += 1
        x1 = x1 + 1
        sum_call += int(z1)   
        
    action_preflop_raises = preflop_.group(1).count('raises')    
    action_preflop_calls = preflop_.group(1).count('calls')
    action_preflop_allin = preflop_.group(1).count('all-in')
    return action_preflop_raises, action_preflop_calls, action_preflop_allin, sum_rais, sum_call

def flop_acti(self):
    flop_ = re.search("FLOP \*{3} \[(.*)\](.*)\*{3} TU", self, re.DOTALL)
    #hand_flop = flop_.group(1)
    x = 0
    i = 0
    max_rais = 0 # максимальный рейз 
    y = 0
    sum_rais = 0 # сумма всех рейзов
    while i != flop_.group(2).count('raises'):
        s = flop_.group(2)
        if s.find('Dealt') == 0 and i == 0:
            y = 10
        if i != 0:
            y = x
        strart = s.find('to ', y)
        x = s.find('\n', strart)
        max_rais= s[strart+3:x]
        if max_rais.find(' ') > 0:
            max_rais = s[strart+3:x-14]
        i += 1
        x = x + 1
        sum_rais += int(max_rais)
        
    z1 = 0
    i1 = 0
    x1 = 0
    sum_call = 0
    
    while i1 != flop_.group(2).count('calls'):
        s = flop_.group(2)
        strart = s.find('calls ',x1)
        x1 = s.find('\n', strart)
        z1= s[strart+6:x1]
        if z1.find(' ') > 0:
            z1 = s[strart+6:x1-14]
        i1 += 1
        x1 = x1 + 1
        sum_call += int(z1)
    
    if flop_.group(2).count('bets'):
        s = flop_.group(2)
        start = s.find('bets ')
        x2 = s.find('\n', start+1)
        z2 = s[start+5:x2]
        if z2.find(' ') > 0:
            z2 = s[start+5:x2-14]
            bets = int(z2)
        bets = int(z2)    
    else:
        bets = 0   
    
    action_flop_bets = flop_.group(2).count('bets') # колличесвто
    action_flop_raises= flop_.group(2).count('raises')
    action_flop_allin = flop_.group(2).count('all-in') 
    action_flop_calls = flop_.group(2).count('calls')
    return action_flop_bets, action_flop_raises, action_flop_calls, action_flop_allin, bets, sum_call, sum_rais

def turn_acti(self):
    turn_ = re.search("TURN \*{3} \[.*\] \[(.*)\](.*)\*{3} RI", self, re.DOTALL)
    #hand_turn = turn_.group(1)
    
    x = 0
    i = 0
    max_rais = 0 # максимальный рейз 
    y = 0
    sum_rais = 0 # сумма всех рейзов
    while i != turn_.group(2).count('raises'):
        s = turn_.group(2)
        if s.find('Dealt') == 0 and i == 0:
            y = 10
        if i != 0:
            y = x
        strart = s.find('to ', y)
        x = s.find('\n', strart)
        max_rais= s[strart+3:x]
        if max_rais.find(' ') > 0:
            max_rais = s[strart+3:x-14]
        i += 1
        x = x + 1
        sum_rais += int(max_rais)
        
    z1 = 0
    i1 = 0
    x1 = 0
    sum_call = 0
    while i1 != turn_.group(2).count('calls'):
        s = turn_.group(2)
        strart2 = s.find('calls ',x1)
        x1 = s.find('\n', strart2)
        z1= s[strart2+6:x1]
        if z1.find(' ') > 0:
            z1 = s[strart2+6:x1-14]
        i1 += 1
        x1 = x1 + 1
        sum_call += int(z1)
    
    if turn_.group(2).count('bets'):
        s = turn_.group(2)
        start = s.find('bets ')
        x1 = s.find('\n', start+1)
        z2 = s[start+5:x1]
        if z2.find(' ') > 0:
            z2 = s[start+5:x1-14]
            bets = int(z2)
        bets = int(z2)    
    else:
        bets = 0    
    
    action_turn_bets = turn_.group(2).count('bets')
    action_turn_raises= turn_.group(2).count('raises')
    action_turn_allin = turn_.group(2).count('all-in')
    action_turn_calls = turn_.group(2).count('calls')
    return action_turn_bets, action_turn_raises, action_turn_calls, action_turn_allin, bets, sum_call, sum_rais

def river_acti(self):
    river_ = re.search("RIVER \*{3} \[.*\] \[(.*)\](.*)\*{3} SHOW", self, re.DOTALL)
    #hand_river = river_.group(1)
    
    x = 0
    i = 0
    max_rais = 0 # максимальный рейз 
    y = 0
    sum_rais = 0 # сумма всех рейзов
    while i != river_.group(2).count('raises'):
        s = river_.group(2)
        if s.find('Dealt') == 0 and i == 0:
            y = 10
        if i != 0:
            y = x
        strart = s.find('to ', y)
        x = s.find('\n', strart)
        max_rais= s[strart+3:x]
        if max_rais.find(' ') > 0:
            max_rais = s[strart+3:x-14]
        i += 1
        x = x + 1
        sum_rais += int(max_rais)
        
    z1 = 0
    i1 = 0
    x1 = 0
    sum_call = 0
    while i1 != river_.group(2).count('calls'):
        s = river_.group(2)
        strart = s.find('calls',x1)
        x1 = s.find('\n', strart)
        z1= s[strart+6:x1]
        if z1.find(' ') > 0:
            z1 = s[strart+6:x1-14]
        i1 += 1
        x1 = x1 + 1
        sum_call += int(z1)
    
    if river_.group(2).count('bets'):
        s = river_.group(2)
        start = s.find('bets ')
        x1 = s.find('\n', start+1)
        z2 = s[start+5:x1]
        if z2.find(' ') > 0:
            z2 = s[start+5:x1-14]
            bets = int(z2)
        bets = int(z2)    
    else:
        bets = 0
    
    action_river_bets = river_.group(2).count('bets')
    action_river_raises= river_.group(2).count('raises')
    action_river_allin = river_.group(2).count('all-in')
    action_river_calls = river_.group(2).count('calls')
    return action_river_bets, action_river_raises, action_river_calls, action_river_allin, bets, sum_call, sum_rais

preflop_raisess = []
preflop_callss = [] 
preflop_allins = []
preflop_raisess_sums =[]
preflop_calls_sums =[]
antes = []

flop_betss = []  
flop_raisess = [] 
flop_callss = []
flop_allins = []
flop_chip_betss = []  
flop_chip_callss = []
flop_chip_raisess = [] 


turn_betss = []  
turn_raisess = [] 
turn_callss = []
turn_allins = []
turn_chip_betss = []  
turn_chip_raisess = [] 
turn_chip_callss = []

river_betss = []  
river_raisess = [] 
river_callss = []
river_allins = []
river_chip_betss = []  
river_chip_raisess = [] 
river_chip_callss = []
    
for y in range(iter_SHOW):
    data_n = data.split("\n\n\n")[y] # разбивает сплошной текст на раздачи
    
# ПРЕФЛОП    
    strart = data_n.find('ante')
    x1 = data_n.find('\n', strart)
    ante = data_n[strart+5:x1]
    antes.append(ante)

    if data_n.count('raises'):
        preflop_raisess.append(preflop_acti(data_n)[0])
    else:
        preflop_raisess.append(0)   
    
    if data_n.count('calls'):
        preflop_callss.append(preflop_acti(data_n)[1])
    else:
        preflop_callss.append(0)
        
    if data_n.count('all-in'):
        preflop_allins.append(preflop_acti(data_n)[2])
    else:
        preflop_allins.append(0)
        
    if preflop_acti(data_n)[3] != 0:
        preflop_raisess_sums.append(preflop_acti(data_n)[3])    
    else:
        preflop_raisess_sums.append(0)    
        
    if preflop_acti(data_n)[4] != 0:
        preflop_calls_sums.append(preflop_acti(data_n)[4])
    else:
        preflop_calls_sums.append(0)      
# ФЛОП        
    if data_n.count('bets'):
        flop_betss.append(flop_acti(data_n)[0])
    else:
        flop_betss.append(0)    
    if data_n.count('raises'):
        flop_raisess.append(flop_acti(data_n)[1])
    else:
        flop_raisess.append(0)
    if data_n.count('calls'):
        flop_callss.append(flop_acti(data_n)[2])
    else:
        flop_callss.append(0)
    if data_n.count('all-in'):
        flop_allins.append(flop_acti(data_n)[3])
    else:
        flop_allins.append(0)    
        
    if flop_acti(data_n)[4] != 0:
        flop_chip_betss.append(flop_acti(data_n)[4])
    else:
        flop_chip_betss.append(0) 
        
    if flop_acti(data_n)[5] != 0:
        flop_chip_callss.append(flop_acti(data_n)[5])
    else:
        flop_chip_callss.append(0)    
        
    if flop_acti(data_n)[6] != 0:
        flop_chip_raisess.append(flop_acti(data_n)[6])
    else:
        flop_chip_raisess.append(0)      
        
# ТЁРН        
    if data_n.count('bets'):
        turn_betss.append(turn_acti(data_n)[0])
    else:
        turn_betss.append(0)    
    if data_n.count('raises'):
        turn_raisess.append(turn_acti(data_n)[1])    
    else:
        turn_raisess.append(0)
    if data_n.count('calls'):
        turn_callss.append(turn_acti(data_n)[2])
    else:
        turn_callss.append(0)
    if data_n.count('all-in'):
        turn_allins.append(turn_acti(data_n)[3])
    else:
        turn_allins.append(0)  
        
    if turn_acti(data_n)[4] != 0:
        turn_chip_betss.append(turn_acti(data_n)[4])
    else:
        turn_chip_betss.append(0) 
        
    if turn_acti(data_n)[5] != 0:
        turn_chip_callss.append(turn_acti(data_n)[5])
    else:
        turn_chip_callss.append(0)    
        
    if turn_acti(data_n)[6] != 0:
        turn_chip_raisess.append(turn_acti(data_n)[6])
    else:
        turn_chip_raisess.append(0)    
        
# РИВЕР        
    if data_n.count('bets'):
        river_betss.append(river_acti(data_n)[0])
    else:
        river_betss.append(0)    
    if data_n.count('raises'):
        river_raisess.append(river_acti(data_n)[1])
    else:
        river_raisess.append(0)
    if data_n.count('calls'):
        river_callss.append(river_acti(data_n)[2])
    else:
        river_callss.append(0)
    if data_n.count('all-in'):
        river_allins.append(river_acti(data_n)[3])
    else:
        river_allins.append(0)   
        
    if river_acti(data_n)[4] != 0:
        river_chip_betss.append(river_acti(data_n)[4])
    else:
        river_chip_betss.append(0) 
        
    if river_acti(data_n)[5] != 0:
        river_chip_callss.append(river_acti(data_n)[5])
    else:
        river_chip_callss.append(0)    
        
    if river_acti(data_n)[6] != 0:
        river_chip_raisess.append(river_acti(data_n)[6])
    else:
        river_chip_raisess.append(0)
    
    
# Считаем сколко фишек
s = re.findall('Seat (\d).*chips', data)
s = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s)-1):
    if s[i+1] < s[i]:
        seats.append(length)
        length = 1
    else:
        length += 1                     
seats.append(length)

# ЗАполнЯем словарь данными
for r, table, seat, Board ,preflop_raises, preflop_calls, preflop_allin, flop_bets, flop_raises, flop_calls, flop_allin, \
    turn_bets, turn_raises, turn_calls, turn_allin, river_bets, river_raises, river_calls, river_allin, SHOW_DOWN, preflop_raisess_sum, \
        preflop_calls_sum, ante, flop_chip_bets, flop_chip_calls, flop_chip_raises, turn_chip_bets, turn_chip_raises, turn_chip_calls, river_chip_bets, \
            river_chip_raises, river_chip_calls \
                in zip(result, Tables, seats, Boards, preflop_raisess, preflop_callss, preflop_allins, flop_betss, flop_raisess, flop_callss, 
                       flop_allins, turn_betss, turn_raisess, turn_callss, turn_allins, river_betss, river_raisess, river_callss, river_allins, SHOW_DOWNs, 
                            preflop_raisess_sums, preflop_calls_sums, antes, flop_chip_betss, flop_chip_callss, flop_chip_raisess, turn_chip_betss, turn_chip_raisess,
                               turn_chip_callss, river_chip_betss, river_chip_raisess, river_chip_callss): 

    #data_pd['TypeOfRoom'].append(r[0])
    data_pd['HandId'].append(r[1])
    #data_pd['Tournament'].append(r[2])
    # data_pd['info'].append(r[3])
    data_pd['Level'].append(r[4])
    data_pd['BB'].append(r[5])
    #data_pd['Date'].append(r[6])
    #data_pd['Hour'].append(r[7])
    #data_pd['TimeZone'].append(r[8])
    data_pd['ante'].append(ante)
    #data_pd['Table'].append(table) #table
    data_pd['Players'].append(seat)
    #data_pd['bain'].append(bain)
    #data_pd['HandHiro'].append(handHiro)
    data_pd['Board'].append(Board)
    data_pd['pflop_calls'].append(preflop_calls)
    data_pd['pflop_rais'].append(preflop_raises)
    data_pd['pflop_allin'].append(preflop_allin)
    data_pd['flop_bets'].append(flop_bets)
    data_pd['flop_raises'].append(flop_raises)
    data_pd['flop_calls'].append(flop_calls)
    data_pd['flop_allin'].append(flop_allin)
    data_pd['turn_bets'].append(turn_bets)
    data_pd['turn_raises'].append(turn_raises)
    data_pd['turn_calls'].append(turn_calls)
    data_pd['turn_allin'].append(turn_allin)
    data_pd['river_bets'].append(river_bets)
    data_pd['river_raises'].append(river_raises)
    data_pd['river_calls'].append(river_calls)
    data_pd['river_allin'].append(river_allin)
    data_pd['pflop_chip_rais'].append(preflop_raisess_sum)
    data_pd['pflop_chip_calls'].append(preflop_calls_sum)
    data_pd['flop_chip_bets'].append(flop_chip_bets)
    data_pd['flop_chip_raises'].append(flop_chip_raises)
    data_pd['flop_chip_calls'].append(flop_chip_calls)
    data_pd['turn_chip_bets'].append(turn_chip_bets)
    data_pd['turn_chip_raises'].append(turn_chip_raises)
    data_pd['turn_chip_calls'].append(turn_chip_calls)
    data_pd['river_chip_bets'].append(river_chip_bets)
    data_pd['river_chip_raises'].append(river_chip_raises)
    data_pd['river_chip_calls'].append(river_chip_calls)
    data_pd['SHOW_DOWN'].append(SHOW_DOWN) 
                                    

# Создаём ДатаФрейм
df = pd.DataFrame(data_pd)
#df[['HandId','flop_chip_bets', 'flop_chip_raises', 'flop_chip_calls', 'turn_chip_bets', 'turn_chip_raises', 'turn_chip_calls', 'river_chip_bets', 'river_chip_raises', 'river_chip_calls', 'pflop_rais_sum', 'pflop_calls_sum']]
df.head(17)

100
40
CPU times: total: 141 ms
Wall time: 126 ms


,HandId,Level,ante,BB,Players,Board,SHOW_DOWN,pflop_chip_rais,pflop_chip_calls,flop_chip_bets,...,flop_calls,flop_allin,turn_bets,turn_raises,turn_calls,turn_allin,river_bets,river_raises,river_calls,river_allin
0,228182836411,IX,25,400,3,5h 2h Kc 3h 5c,7h 7s,9842,0,0,...,0,0,0,0,0,0,0,0,0,0
1,228182839161,IX,25,400,3,Jd Qh Qc 6d Kd,9c Tc,0,400,400,...,1,0,0,0,0,0,1,0,1,0
2,228182845929,IX,25,400,3,Td Kh Th 2c Ad,Tc 8s,2483,2083,0,...,0,0,0,0,0,0,0,0,0,0
3,228182848632,IX,25,400,3,3h 6d 7d 8d 6h,Qc 7c,0,29,0,...,0,0,0,0,0,0,0,0,0,0
4,228243482155,III,6,50,9,Td 6s 4c 8c Jd,Kc As,1585,996,0,...,0,0,0,0,0,0,0,0,0,0
5,228244105734,V,15,120,5,4h 5h 8c 5c 7h,Ah Kh,1261,1141,0,...,0,0,0,0,0,0,0,0,0,0
6,228244118113,V,15,120,5,6d 3h 5d 6h 8h,9c 7c,499,158,0,...,0,0,0,0,0,0,0,0,0,0
7,228244311794,VI,20,160,4,Td 6c 4s Ts 5h,5s 5h,320,160,280,...,1,0,0,0,0,0,0,0,0,0
8,228244354022,VI,20,160,4,Qc 4h Kc Td 8d,6d 4d,1330,1170,0,...,0,0,0,0,0,0,0,0,0,0
9,228244452800,VI,20,160,4,5s Tc 3c Ts 9d,Ts Qd,0,240,0,...,0,0,1,0,1,0,0,0,0,0


In [258]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   HandId             17 non-null     object
 1   Level              17 non-null     object
 2   ante               17 non-null     object
 3   BB                 17 non-null     object
 4   Players            17 non-null     int64 
 5   Board              17 non-null     object
 6   SHOW_DOWN          17 non-null     object
 7   pflop_chip_rais    17 non-null     int64 
 8   pflop_chip_calls   17 non-null     int64 
 9   flop_chip_bets     17 non-null     int64 
 10  flop_chip_raises   17 non-null     int64 
 11  flop_chip_calls    17 non-null     int64 
 12  turn_chip_bets     17 non-null     int64 
 13  turn_chip_raises   17 non-null     int64 
 14  turn_chip_calls    17 non-null     int64 
 15  river_chip_bets    17 non-null     int64 
 16  river_chip_raises  17 non-null     int64 
 17 

In [3]:
!pip install pyqtwebengine -v 5.13

Using pip 22.2.2 from C:\Users\artio\anaconda3\lib\site-packages\pip (python 3.9)
     ------------------------------------ 182.7/182.7 kB 283.1 kB/s eta 0:00:00


ERROR: Could not find a version that satisfies the requirement 5.13 (from versions: none)
ERROR: No matching distribution found for 5.13


In [15]:
from flopferret import board_texture

help(board_texture)

Help on module flopferret.board_texture in flopferret:

NAME
    flopferret.board_texture

DESCRIPTION
    # Copyright (C) 2014 Julian Andrews
    # This file is part of Flop Ferret.
    #
    # Flop Ferret is free software: you can redistribute it and/or modify
    # it under the terms of the GNU General Public License as published by
    # the Free Software Foundation, either version 3 of the License, or
    # (at your option) any later version.
    #
    # Flop Ferret is distributed in the hope that it will be useful,
    # but WITHOUT ANY WARRANTY; without even the implied warranty of
    # MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    # GNU General Public License for more details.
    #
    # You should have received a copy of the GNU General Public License

CLASSES
    builtins.dict(builtins.object)
        BoardTexture
    
    class BoardTexture(builtins.dict)
     |  Method resolution order:
     |      BoardTexture
     |      builtins.dict
     |      bui

In [11]:
help(calculate)

NameError: name 'calculate' is not defined

## Преобразуем данные

In [ ]:
date = data_pd['Date']
time = data_pd['Hour']

dataf["Date"] = pd.to_datetime(data_pd['Date'])
dataf["Time"] = pd.to_timedelta(data_pd['Hour'])

dataf["DateTime"] = dataf["Date"] + dataf["Time"]
dataf["DateTime"]

# filter by single day
dataf_m = dataf[dataf['DateTime'].dt.strftime('%d') == '06']

dataf.Tournament = dataf.Tournament.astype('int64')
dataf = dataf.astype({'HandId': np.int64, 'Tournament': np.int64})

In [ ]:
dataf_m

In [ ]:
nov_mar_series = pd.Series(pd.date_range("2013-11-15", "2014-03-15"))
#create timestamp without year
nov_mar_no_year = nov_mar_series.map(lambda x: x.strftime("%m-%d"))
nov_mar_no_year

In [ ]:
# #add a yearless timestamp to the dataframe
# df["no_year"] = df['nov_mar_series'].map(lambda x: x.strftime("%m-%d"))
# no_year_mask = df['no_year'].isin(nov_mar_no_year)
# df[no_year_mask]

## Фильтруем турниры по числу рук выше среднего (> mean)

In [ ]:
data_filter = dataf[['Tournament', 'HandId','DateTime', 'Time', 'HandHiro', 'Board']]
data_filter.head()

In [ ]:
# data_filter.groupby(['Tournament']).agg({'HandId': "count"}).reset_index()

In [ ]:
#  Фильтруйте value_counts с помощью isin  https://softhints.com/pandas-how-to-filter-results-of-value_counts/
df = data_filter[data_filter['Tournament'].isin(data_filter['Tournament'].
                                                 value_counts()[data_filter['Tournament'].value_counts()>100].index)]
df

In [ ]:
#  Используйте group by и lambda для имитации фильтра для value_counts()
#df = data_filter.groupby('Tournament').filter(lambda x: len(x) > (data_filter['Tournament'].value_counts().mean()))


In [ ]:
df.Tournament.value_counts()

In [ ]:
# Prepare Data
df_plot = df.groupby('Tournament').size()

# Make the plot with pandas
df_plot.plot(kind='pie', subplots=True, figsize=(8, 8))
plt.title("Колличество турниров")
plt.ylabel("")
plt.show()

In [ ]:
# объеденяем руку и борд, выводим название комбинации

# df['hand'] = df['HandHiro'] + " " + df['Board']
# # hand = df['hand'].values[7].split()
# # print(hand)

# hand_ev = [eval7.Card(s) for s in (df['hand'].values[7].split())]
# rang = eval7.evaluate(hand_ev)   # ранг комбинации
# # print(rang)

# eval7s = eval7.handtype(rang)  #  название комбинации

# df['rang'] = eval7.evaluate(hand_ev)
# df['eval7s'] =  eval7.handtype(rang)

In [ ]:
# def new_type(row):
#     if row.traffic_source.lower() in ('yandex', 'google'):
#         return 'organic'
#     elif row.traffic_source.lower() in ('paid', 'email'):
#         if row.region.lower() == 'russia':
#             return 'ad'
#         else:
#             return 'other'

In [ ]:
# log['source_type'] = log.apply(new_type, axis = 1)
# log.head(12)

In [ ]:
# log.loc[log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic' 

In [ ]:
df['hand_rang'] = df['HandHiro'] + " " + df['Board']
df['hand_rang']

print(type(df['hand_rang']), type(df['hand_rang'].values[7].split()))
df['hand_rang'].values[0].split()

In [ ]:
# df['hand_rang'].to_list()
# df['hand_rang'].tolist()
# np.array(df['hand_rang'].tolist())
# Ls = list(df['hand_rang'])
ls = df['hand_rang'].str.split()
ls

In [ ]:
# напишем функцию получения ранг комбинации

def evaluate_(hand):
    """
    На вход получает карты, на выход 
    число ранг по eval7
    """
    hand_ev = [eval7.Card(s) for s in (hand)]
    return eval7.evaluate(hand_ev),hand_ev

a = ('Jh', 'Td', '8h', '8c', '7c', '3h', '6d')
evaluate_(a)

In [ ]:
# напишем функцию получения названия ранга

def handtype_(rang):
    """
    На вход получает ранг, на выход 
    название комбинации
    """
    return eval7.handtype(rang)

handtype_(17209424)

In [ ]:
# Применим функцию к столбцу hand  и запишем в новый столбец rang_namber

df['rang_namber'] = df['hand_rang'].str.split().apply(evaluate_)
df['handtypes'] = df['rang_namber'].apply(handtype_)
df[['rang_namber', 'handtypes','HandHiro', 'Board']].head(20)
# df['filter'] = df.loc[df.handtypes.str.contains('Two Pair', 'Trips', 'Straight', 'Full House', 'Quads', 'Straight Flush'), 'rang_namber']

In [ ]:
# df.sort_values(by=['handtypes', 'rang_namber'])

In [ ]:
# df.groupby('handtypes')['rang_namber'].value_counts()

### Создадим сводную таблицу, что-бы проверить наличие аномалий

In [ ]:
# filter by single day
dataf_m = dataf[dataf['DateTime'].dt.strftime('%d') == '06']

In [ ]:
df[['Tournament', 'handtypes', 'rang_namber']].head()  # применим фильтер по столбцам

In [ ]:
pd.pivot_table(df, index=["Tournament", "handtypes"])

In [ ]:
# явно определив столбцы, которые нам нужны,
pd.pivot_table(df,
               index=["Tournament", "handtypes"],
               values=["rang_namber"],
               aggfunc= len)   # aggfunc=[np.mean, len])

In [ ]:
df.Tournament.value_counts()

In [ ]:
# df['Time'].value_counts()

In [ ]:
# Prepare Data
df_plot = df.groupby('handtypes').size()

# Make the plot with pandas
df_plot.plot(kind='area', subplots=True, figsize=(8, 8))
plt.title("Колличество комбинаций")
plt.ylabel("")
plt.show()

In [ ]:
# df['DateTime'] = df['DateTime'][(df['DateTime'] > '2018-10-01 20:00:00') & (df['DateTime'] < '2022-10-02 00:00:00')]
# df['DateTime']


In [ ]:
# отфильтруем комбинации "старшая карта" и "пара"
df['no_par'] = df['rang_namber'][df['rang_namber'] > 50000000] #[((df['rang_namber'] > 50000000) & (df['rang_namber'] < 60000000))]
df = df.dropna()
df[['Time', 'no_par', 'handtypes']].head(10)
# df['filter_Two Pair'] = df['handtypes'][df['handtypes'] == 'Two Pair']
# df['filter_Two Pair']

In [ ]:
df.isnull().sum()

In [ ]:
# преобразуем время во временной ряд

In [ ]:


idx = df['Time']
 # datetime.timedelta(seconds=24*60*60).total_seconds()

# second = df['DateTime'].dt.total_seconds()
# second

# idx = pd.date_range(df['DateTime'], periods=5, freq="H")
# idx
tim = pd.Series(range(len(idx)), index=idx)

# ts.resample("2H").mean()
tim

In [ ]:
# df = df.set_index("Time")
# df

In [ ]:

import matplotlib.pyplot as plt
 
# Using a inbuilt style to change
# the look and feel of the plot
plt.style.use("classic")
 
# setting figure size to 12, 10
plt.figure(figsize=(12, 10))
 
# Labelling the axes and setting
# a title
plt.xlabel("Time")
plt.ylabel("Values")
plt.title("Sample Time Series Plot")
 
# plotting the "A" column alone
plt.plot(df["handtypes"])

In [ ]:
df.plot(subplots=True, figsize=(12, 15))

In [ ]:
# # # idx = pd.date_range('2019-01-01', periods=48, freq='H')
# # x = pd.Series(range(len(idx)), index=idx)
# y = tim
# # add 'hour' and 'day' columns in the dataframe
# d = {'x': x, 'y': y, 'hour': idx.hour, 'day': idx.day}
# df = pd.DataFrame(data=d)
# # use 'hour' as x axis to plot, and 'day' as marker color
# df.plot.scatter(x='hour', y='y', c='day', colormap='rainbow')
# plt.show()

In [ ]:
data.dtypes

In [ ]:
# установитьdatetimeОбъект в качестве индекса кадра данных
data = data.set_index('DateTime')
data.head()

In [ ]:

# data.plot(grid=True)

data['filter'].plot(kind='bar')

# import matplotlib.pyplot as plt
# from matplotlib.dates import DateFormatter
# # Группировка за месяц
# df_month = data.resample("M").mean()

# fig, ax = plt.subplots(figsize=(10, 6))
# ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
# ax.bar(
#     df_month['2018':].index, 
#     df_month.loc['2016':, "filter"], 
#     width=25, align='center')

In [ ]:

data.plot(kind='scatter', 
        x='DateTime', 
        y='filter', 
        title='Связь между жёсткостью воды и средней годовой смертностью')